In [ ]:
# Make sure you have all the packages below installed in your python installation.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline  
import sys

In [ ]:
# To be able to run the lines below, you need to have installed allensdk. If you don't have it 
# yet, go to where you have your python installation (e.g., the anaconda directory) and type:
# pip install allensdk
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
boc =  BrainObservatoryCache(manifest_file='boc/manifest.json')

In [ ]:
from allensdk.brain_observatory.drifting_gratings import DriftingGratings
# 541512399
data_set = boc.get_ophys_experiment_data(502376461)
dg = DriftingGratings(data_set)

In [ ]:
from allensdk.core.cell_types_cache import CellTypesCache
import pandas as pd
# Instantiate the CellTypesCache instance.  The manifest_file argument
# tells it where to store the manifest, which is a JSON file that tracks
# file paths.  If you supply a relative path (like this), it will go
# into your current working directory
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

# this saves the NWB file to 'cell_types/specimen_464212183/ephys.nwb'
cell_specimen_id = 464212183
data_set = ctc.get_ephys_data(cell_specimen_id)
cell_specimens =  pd.DataFrame(ctc.get_cell_specimens())

In [ ]:
def get_dff_traces_and_stim_table(cell_specimen_id, stimulus):  
    import pandas as pd
    print("Downloading cell_specimens. This will take a while the first time it is run.")
    cell_specimens = pd.DataFrame(boc.get_cell_specimens())
    expt_id = cell_specimens[cell_specimens.cell_specimen_id==cell_specimen_id].experiment_container_id.values[0]
    session_id = boc.get_ophys_experiments(stimuli=[stimulus],experiment_container_ids=[expt_id])[0]['id']
    print("Opening NWB file for ophys session. This will take several minutes if the file isn't downloaded yet.")
    data_set = boc.get_ophys_experiment_data(session_id)
    print("All Done")
    timestamps, dff = data_set.get_dff_traces(cell_specimen_ids=[cell_specimen_id])
    dff_trace = dff[0,:]
    stim_table = data_set.get_stimulus_table(stimulus)
    return (timestamps, dff_trace, stim_table)


In [ ]:
# You can get a cell by running the function above for a certain cell id, e.g., 541512399
cell = 541512399
timestamps, dff_trace, stim_table = get_dff_traces_and_stim_table(cell, 'drifting_gratings')
cell_response= np.zeros((len(stim_table),3))
for k in range(len(stim_table)):
    cell_response[k,0] = stim_table.orientation[k]
    cell_response[k,1] = stim_table.temporal_frequency[k]
    cell_response[k,2] = dff_trace[stim_table.start[k]:stim_table.end[k]].mean()
        
all_ori = np.unique(cell_response[:,0])
orivals = all_ori[np.isfinite(all_ori)]

tuning = np.empty(len(orivals))
for i, ori in enumerate(orivals):
    trials = np.where(cell_response[:,0]==ori)[0]
    tuning[i] = cell_response[trials,2].mean()

tfvals = np.unique(cell_response[:,1])
tfvals = tfvals[np.isfinite(tfvals)]

tuning_tf = np.empty(len(tfvals))
for i, tf in enumerate(tfvals):
    trials = np.where(cell_response[:,1]==tf)[0]
    tuning_tf[i] = cell_response[trials,2].mean()

In [ ]:
# Create a plot for the cell you are studying
fig=plt.figure()
# Create grispec object and define each subplot
gs = gridspec.GridSpec(2, 4)
gs.update(wspace=0.9, hspace=0.9) # better spacing
ax0 = plt.subplot(gs[0, :]) 
ax1 = plt.subplot(gs[1, :]) 

ax0.plot(orivals, tuning, 'o-')
ax0.set_xlim(0,360)
ax0.set_xticks(orivals)
ax0.set_xlabel("Direction (deg)", fontsize=12)
ax0.set_ylabel("DF/F", fontsize=12)

ax1.plot(tfvals, tuning_tf, 'o-')
ax1.set_xlim(0,20)
ax1.set_xticks(tfvals)
ax1.set_xlabel("temporal frequency (Hz)", fontsize=12)
ax1.set_ylabel("DF/F", fontsize=12)

ax0.set_title('Flourence v. Orientation')
ax1.set_title('Flourence v. Frequency')
fig.suptitle('Tuning Curves for cell id'+str(cell),fontsize=16)